# Unit 1 - Cost of Teacher

Group members:

- Ole Magnus Laerum
- Piotr Franc

## Task description
Use the dataset BostonHousing2 (target value is cmedv) with the following train1/train2/test partition. Use the dataset train1 to explore learning methods and techniques in a way that you want to reduce the amount of data used for training without compromising the performance of the model too much. Once the best methodology is selected, use it with train2 to show a plot of performance (RMSE) vs instances employed measured with the test partition. Grow this plot incrementally (i.e, add new instances to the already used instances) Select the instances of train2 without using the class label of the instances already not employed.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

text = """# Load Boston Housing dataset
boston = fetch_openml(name='boston', as_frame=True)
boston_df = boston.frame
boston_df.rename(columns={'medv': 'cmedv'}, inplace=True)

# Drop unwanted column
boston_df.drop(columns=['chas'], inplace=True)

# Partitioning the dataset
np.random.seed(280)
tr1, temp = train_test_split(boston_df, test_size=0.6, random_state=280)
tr2, te = train_test_split(temp, test_size=0.5, random_state=280)"""

print("Hello World")


Hello World
